In [1]:
import pickle as pkl
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

/home/berk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
own_data = pd.read_csv("/home/berk/VS_Projects/simglucose/exercise/notebooks/combined_adolescent.csv")

In [3]:
type(own_data)
own_data.describe()
own_data.head()

,no,Action,CHO,reward,BG,CGM,RI,LBGI,HBGI,MRI
0,0,0.000008,0.0,1.0,155.583498,164.647835,3.668147,0.0,3.668147,0.595842
1,1,0.000000,0.0,1.0,154.942206,163.893527,3.574990,0.0,3.574990,0.553341
2,2,0.000000,0.0,1.0,154.394367,163.081224,3.496061,0.0,3.496061,0.518141
3,3,0.000000,0.0,1.0,153.921872,162.242153,3.428477,0.0,3.428477,0.488613
4,4,0.000000,0.0,1.0,153.511040,161.412521,3.370085,0.0,3.370085,0.463569


In [4]:
own_data_value = own_data.values
print(own_data_value)

[[0.00000000e+00 8.33333300e-06 0.00000000e+00 ... 0.00000000e+00
  3.66814722e+00 5.95842080e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  3.57498975e+00 5.53340630e-01]
 [2.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  3.49606116e+00 5.18141230e-01]
 ...
 [9.86760000e+04 0.00000000e+00 0.00000000e+00 ... 3.05052985e-02
  0.00000000e+00 2.70089064e+00]
 [9.86770000e+04 0.00000000e+00 0.00000000e+00 ... 2.91084648e-02
  0.00000000e+00 2.68540383e+00]
 [9.86780000e+04 0.00000000e+00 0.00000000e+00 ... 2.77909350e-02
  0.00000000e+00 2.67049472e+00]]


In [5]:
actions_own_data = own_data['Action'].values
states_own_data = own_data['BG'].values
reward_own_data = own_data['reward'].values
terminals_own_data = np.full(len(actions_own_data), False)
next_state_own_data = own_data['BG'].values[1:]


In [6]:
next_state_own_data = np.append(next_state_own_data, next_state_own_data[-1])

In [7]:
terminals_own_data[-1] = True

In [8]:
data_dict = {'observations': states_own_data, 'next_observations':next_state_own_data, 
             'actions': actions_own_data, 'rewards': reward_own_data, 'terminals': terminals_own_data}

In [24]:
seq_length = 4800
final_data = []
for idx in range(0,len(states_own_data),seq_length):
    data_dict = {'observations': np.array([states_own_data[idx:seq_length+idx]]), 'next_observations':np.array([next_state_own_data[idx:seq_length+idx]]), 
                'actions': np.array([actions_own_data[idx:seq_length+idx]]), 'rewards': np.array([reward_own_data[idx:seq_length+idx]]), 
                'terminals': np.array([terminals_own_data[idx:seq_length+idx]])}
    final_data.append(data_dict)
    data_dict = {}

In [25]:
(final_data[0]['terminals'][:,-1])

array([False])

In [27]:
for i in range(len(final_data)):
    final_data[i]['observations']=final_data[i]['observations'].squeeze()
    final_data[i]['next_observations']=final_data[i]['next_observations'].squeeze()
    final_data[i]['rewards']=final_data[i]['rewards'].squeeze()
    final_data[i]['actions']=final_data[i]['actions'].squeeze()
    final_data[i]['terminals']=final_data[i]['terminals'].squeeze()
    final_data[i]['terminals'][-1]= True

In [30]:
# Save the dictionary to a pickle file
with open('./PPO_traj_4800_step.pkl', 'wb') as file:
    pkl.dump(final_data, file)

In [31]:
with open('./PPO_traj_4800_step.pkl', 'rb') as file:
    final_data = pkl.load(file)

In [32]:
np.shape(final_data[0]['terminals'])

(4800,)

In [33]:
np.shape(final_data[1]['terminals'])

(4800,)